In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
# Define cubic SE18, find and classify the lines and compute the 45 tritangent planes

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE18 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : i_rad3*c})

CPU times: user 3.12 s, sys: 3.47 ms, total: 3.12 s
Wall time: 3.12 s


In [6]:
# Find simmetries

In [7]:
%time adm_SE18 = SE18.find_admissible_projectivities()
len(adm_SE18)

CPU times: user 801 ms, sys: 60.3 ms, total: 862 ms
Wall time: 9.51 s


648

In [8]:
%time simm_SE18 = SE18.find_simmetries(adm_SE18)
len(simm_SE18)

CPU times: user 848 ms, sys: 96.6 ms, total: 945 ms
Wall time: 1.23 s


648

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
%%time
Eck_points_perms = []
for proj in simm_SE18:
    Eck_points_perms.append(Permutation(SE18.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

CPU times: user 7.36 s, sys: 83.6 ms, total: 7.45 s
Wall time: 8.35 s


(648, '(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2')

In [11]:
# Sage's description is not too helpful, but we actually know that the group 
# is isomorphic to (C3 X C3 X C3) : S4
# What is this S4? It is the group of permutations of the four planes containing a C9 configuration
# i.e. 12 coplanar lines passing through 9 Eckardt points

In [18]:
# Build all the lines through three Eckardt points that are not contained is SE18
lines_three_eck = []
for i in range(16):
    for j in range(i+1, 17):
        for k in range(j+1, 18):
            point1 = SE18.eckardt_points[i]
            point2 = SE18.eckardt_points[j]
            point3 = SE18.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_two_planes_containing_line(point1, point2)
                lines_three_eck.append(Line(planes,[point1, point2], line12_pl))
len(lines_three_eck)

42

In [19]:
# Find all planes containing the lines through three Eckardt points
planes = []
for line in lines_three_eck:
    for line2 in line.get_all_lines_incident_to_self(lines_three_eck):
        planes.append(line.get_plane_containing_another_incident_line(line2))
planes = list(set(planes))
len(planes)

31

In [20]:
# The four planes we are looking for are those containing exactly 12 lines on them
c9_planes = [plane for plane in planes if [line.is_on_plane(plane) for line in lines_three_eck].count(True) == 12]
c9_planes

[x + (-alpha - 1)*y - t,
 x + (-1/4*alpha - 1/4)*y + (1/4*alpha + 1/4)*z,
 x + (alpha - 1)*y - t,
 x + (1/4*alpha - 1/4)*y + (-1/4*alpha + 1/4)*z]

In [12]:
nor_subs = Eck_points_perms_group.normal_subgroups()

In [13]:
# all elements of H have order 3
H = [el for el in nor_subs if el.order()==27][0]
H.structure_description(), list(set([el.order() for el in H if el.cycle_tuples() != []])) == [3]

('C3 x C3 x C3', True)

In [14]:
S4 = Eck_points_perms_group.quotient(H)
S4.is_isomorphic(SymmetricGroup(4))

True

In [15]:
# We now want explicitly a subgroup isomorphic to S4

In [16]:
%time all_subgroups = Eck_points_perms_group.subgroups()

CPU times: user 50.4 s, sys: 120 ms, total: 50.5 s
Wall time: 50.6 s


In [17]:
subS4 = [el for el in all_subgroups if el.is_isomorphic(S4)][0]
gens_indices = [Eck_points_perms.index(gen) for gen in subS4.minimal_generating_set()]

In [21]:
# We see that the generators of S4 permute these four planes

In [22]:
simms = [simm_SE18[index] for index in gens_indices]

In [23]:
for simm in simms:
    coord_change = change_coord(simm)
    new_planes = [plane.subs(coord_change) for plane in c9_planes]
    for new_plane in new_planes:
        print(bool([matrix([plane_coefficients(old_plane), plane_coefficients(new_plane)]).minors(2) == [0,0,0,0,0,0] for old_plane in c9_planes]))

True
True
True
True
True
True
True
True


In [24]:
# Study how the simmetries permute the 27 lines

In [25]:
%%time
lines_perms = []
for simm in simm_SE18:
    lines_perms.append(Permutation(SE18.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

CPU times: user 56 s, sys: 312 ms, total: 56.3 s
Wall time: 57.2 s


(648, '(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2')